![Alt text](images/banner.png)

## Connection and Data Validation Notebook

### Load the Credentials

These environment variables are automatically set in WS Pipelines and are needed to access various services.

In [ ]:
import os
TOKEN = os.getenv("USER_ACCESS_TOKEN")

In [ ]:
## Imports

In [4]:
from botocore.client import Config
from sklearn.model_selection import train_test_split
from dataclasses import dataclass
import tensorflow_data_validation as tfdv
import numpy as np
import pandas as pd
from ibm_watson_studio_pipelines import WSPipelines
import warnings


warnings.filterwarnings("ignore")

2023-10-23 07:24:33.933101: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-23 07:24:33.933161: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-23 07:24:33.933204: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-23 07:24:35.469552: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### Loading Variables and Utils from common python file

In this section we load the variables and functions from the common python file. This file contains the variables and functions that are common to all the notebooks in this project.

In [ ]:
import vars_and_utils as vars_and_utils

## Load the Training Data 

this will check if the training data exists within a defined db2 table. If it does not exist, it will load the data from the web and store it in the project space as a .csv file.

In [14]:
gcr_df = vars_and_utils.load_german_credit_risk_data()

## Encode for ease of use with OpenScale
gcr_df['Risk'] = gcr_df['Risk'].map({'Risk':1,'No Risk':0})
gcr_df.head()

NameError: name 'training_file_path' is not defined

## Data Validation 

In [ ]:
@dataclass
class Datavalidation:
    """
    
    Data Validation Class
    
    """
    dataframe : pd.DataFrame
    mask_per :int
    
    
    def split_data(self,seed=32):
        """
        Split Data into Train and Test Splits
        
        """
        np.random.seed(seed)
        mask = np.random.rand(len(self.dataframe)) <= self.mask_per
        training_data = gcr_df[mask]
        testing_data = gcr_df[~mask]

        print(f"No. of training examples: {training_data.shape[0]}")
        print(f"No. of testing examples: {testing_data.shape[0]}")
        
        return training_data, testing_data
    
    # TODO: Replace with Db2/fileystem
    def save_data_in_filesystem(self,df,filename):
        """
        Save Data in Filesystem

        Passed filename should involve path

        """
        try:
            df.to_csv(filename,index=False)
            print(f"File {filename} persisted successfully")
        except Exception as e:
            print(e)
            print(f"File serialization for {filename} failed")
    
    def generate_statistics(self,df):
        """
        
        Generate Statistics on a given Dataframe
        
        """
        train_stats = tfdv.generate_statistics_from_dataframe(df)
        tfdv.visualize_statistics(train_stats)
        return train_stats
    
    def inferSchema(self,stats):
        
        """
        InferSchema on a given Dataframe
        
        """
        schema = tfdv.infer_schema(statistics=stats)
        tfdv.display_schema(schema=schema)
        return schema
    
    def compare_statistics(self,lhs,rhs):
        """
        
        Compare Statistics between a test dataframe and reference Schema
        
        """
        # Compare evaluation data with training data
        tfdv.visualize_statistics(lhs_statistics=lhs, rhs_statistics=rhs,
                                  lhs_name='TEST_DATASET', rhs_name='TRAIN_DATASET')
        
        
    def check_for_anomalies(self,testable_stats,ref_schema):
        """
        
        Check for any anomalies based on statistics and schema and values
        
        """
        anomalies = tfdv.validate_statistics(statistics=testable_stats, schema=ref_schema)
        tfdv.display_anomalies(anomalies)
        if len(anomalies.anomaly_info.items()) > 0:
            logger.error("Anomalies found in dataset...")
            logger.error(str(self.anomalies.anomaly_info.items()))
            return True
        else:
            return False

###  Split Data into Train and Eval Splits to Check for Consistency

In [ ]:
classvalidate = Datavalidation(dataframe=gcr_df,mask_per=0.8) 

training_data, testing_data = classvalidate.split_data()

## Generate Training Stats on both Splits

In [ ]:
train_stats = classvalidate.generate_statistics(training_data)
test_stats = classvalidate.generate_statistics(testing_data)

## Infer Data Schemas

In [ ]:
train_schema = classvalidate.inferSchema(train_stats)
test_schema = classvalidate.inferSchema(test_stats)

## Compare Eval and Train Data 

In [ ]:
classvalidate.compare_statistics(lhs=test_stats,rhs=train_stats)

## Check For Data Anomalies 

### Check eval data for errors by validating the eval data stats using the previously inferred schema.

In [ ]:
anomaly_status = classvalidate.check_for_anomalies(test_stats,train_schema)
anomaly_status

## Save Train and Test Data for Data Preparation Stage

In [ ]:
# TODO: Replace with Db2/fileystem
if not anomaly_status:
    classvalidate.save_data_in_filesystem(df=training_data,filename=vars_and_utils.train_data_path)
    classvalidate.save_data_in_filesystem(df=testing_data,filename=vars_and_utils.test_data_path)

## Check if the validation steps were successful
This checks if anomalies were found and if the data was successfully split into train and eval splits and stored as files.

In [ ]:
def validation_successfull(train_data_path, test_data_path):
    if anomaly_status: # no anomalies
        return False
    elif not os.path.exists(train_data_path): # train data file exists
        return False
    elif not os.path.exists(test_data_path): # test data file exists
        return False
    else:
        print ("validation of the data successfull")
        return True
    
validation_successfull(vars_and_utils.train_data_path, vars_and_utils.test_data_path)

## Register the output variables for the next pipeine stage
every notebook outputs a "was_successful" boolean variable. The logic behind this is different for every notebook and can be altered to fit the needs of the project.
If needed additional variables can be created here but they also need to registered as output variables in the Watson Pipelines UI.

In [ ]:
validation_params = {}
validation_params['was_succesfull'] = validation_successfull(vars_and_utils.train_data_path, vars_and_utils.test_data_path)

pipelines_client = WSPipelines.from_token(TOKEN)
pipelines_client.store_results(validation_params)